In [101]:
import pandas as pd
import numpy as npi

In [102]:
df = pd.read_csv("cleaner_transformed_data_Rachael_with_no_spaces_in_column_names.csv")

In [103]:
df.columns

Index(['Unnamed: 0', 'channelDescription', 'channelJoinedDate',
       'channelTotalVideos', 'channelTotalViews', 'channelUsername',
       'commentsCount', 'date', 'duration', 'id', 'isChannelVerified', 'likes',
       'numberOfSubscribers', 'order', 'text', 'title', 'url', 'viewCount',
       'likes_per_subscriber', 'comments_per_subscriber',
       'views_per_subscriber', 'duration_in_seconds', 'datetime',
       'datetime_date'],
      dtype='object')

In [131]:
df2 = df[["channelUsername","date","title","likes","viewCount", "numberOfSubscribers", "likes_per_subscriber", "views_per_subscriber", "duration_in_seconds"]]

In [132]:
df2["days_since_2024_01_01"] = (pd.to_datetime(df2["date"]) - pd.to_datetime("2024-01-01").tz_localize("UTC")).dt.days

/var/folders/h5/lx6x17415p370njcbygnxm200000gn/T/ipykernel_31091/4290197809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["days_since_2024_01_01"] = (pd.to_datetime(df2["date"]) - pd.to_datetime("2024-01-01").tz_localize("UTC")).dt.days


In [133]:
df2 = df2.query("days_since_2024_01_01 <= 366")

In [134]:
#Here I was trying to see if there were any titles that were repeated. 
#What I noticed was that certain content creators have their own theme/running series of videos that they label with specific titles.
#AtarahMayhew uses "I 👅 U" as the title for any video where she swallows a piece of paper after doing her makeup
#AtarahMayhew has a series of "CUTE ✅ or FAIL? ❌ Tiktok Filters Pick My Makeup 😱"
#Julia.P uses "Dior lip maximizer", "Kiko Milano lip gloss", 'New Dior lipstick', 'Fenty beauty lip oil','Queen Musia lipstick'! 
# where shes just showing the packaging with viral transition sounds or songs playing over the audio
#alot of swatches, but sort of low-effort, with no description etc
#nats_tutorials seems to have a running series of "testing a weird ____" which results in several videos of the same title
#lifebyjessica1 has a running series of "dollar store finds"
#lindseyteaches has a running series of teacher hairstyles of the day
#SofiaSeesBeauty was spamming her new brush set/using the title as an advertisement 
#blueyoobin has a 5 step skincare series

#the main offenders for title repetition (they have more than one series running) are: AtarahMayhew, Julia.P, nats_tutorials

counts = df2['title'].value_counts()
with pd.option_context("display.max_rows", None):
    display(counts)


title
I 👅 U                                                                                                   9
Dior lip maximizer                                                                                      7
Testing a Weird Concealer Applicator                                                                    7
Kiko Milano lip gloss                                                                                   7
CUTE ✅ or FAIL? ❌ Tiktok Filters Pick My Makeup 😱                                                       7
ASMR Makeup Tutorial! ✨GRWM💄                                                                            6
Dior single eyeshadow                                                                                   6
Testing a Weird Beauty Sponge                                                                           5
NYX slick click fat oil                                                                                 5
Sofia Sees Beauty x ESUM is here! Shop m

In [135]:
#I was wondering if there were keywords I wasn't thinking of - just from some bias I have, so instead I search for all words used in every title
#And order them by frequency. 
#I then tried to remove the obvious articles and conjunctions, but somes still come through, maybe some blank space or smthn subtle? 

word_list = []
list_to_ignore = ["for","the","a","me","my","in","for","but","of","this","that","The","with","is","you","&","your","it","do","be","by","so","What","what","With","all","i","I","if","Why","why"]
list_to_ignore = set([word.lower() for word in list_to_ignore])
for _, row in df.iterrows():
    new_words = row["title"].split()
    word_list = word_list + [word.lower() for word in new_words if word not in list_to_ignore]

with pd.option_context("display.max_rows", None):
    display(pd.Series(word_list).value_counts())

#makeup                                             2200
makeup                                              1825
#shorts                                             1818
to                                                  1738
#skincare                                           1511
lip                                                 1361
#beauty                                             1111
skin                                                 981
new                                                  969
the                                                  963
i                                                    882
part                                                 879
skincare                                             833
#kbeauty                                             826
how                                                  802
beauty                                               790
on                                                   678
and                            

In [136]:
#streamlining the "presence of a substring" function Rachael was implementing earlier - Jo

def compare_means(df, thing_to_check_for, thing_to_take_average_of, where_to_look="title"):
    print(f"Comparing means of {thing_to_take_average_of} with and without {thing_to_check_for}.")
    idxs = (df[where_to_look].str.count(thing_to_check_for) > 0)
    print(df.loc[idxs, thing_to_take_average_of].mean())
    print(df.loc[~idxs, thing_to_take_average_of].mean())

def get_mean_with_word(df, thing_to_check_for, thing_to_take_average_of, where_to_look="title"):
    idxs = (df[where_to_look].str.count(thing_to_check_for) > 0)
    return df.loc[idxs, thing_to_take_average_of].mean()

def get_mean_without_word(df, thing_to_check_for, thing_to_take_average_of, where_to_look="title"):
    idxs = (df[where_to_look].str.count(thing_to_check_for) == 0)
    return df.loc[idxs, thing_to_take_average_of].mean()


In [137]:
# 12. Identify popular video topics/formats and see if videos that cover these topics perform better than average. 
# For example, one popular video format is "speed reviews."
#"speed reviews", "haul", "dupe" , "GRWM"  etc


# Create a dataframe
schema = pd.DataFrame()

words =["dupe","cheap","drugstore"]# add more words
column_to_take_average_of = "likes_per_subscriber"
where_to_look_for_word = "title"

#added color vs colour to see if there was an american bias
#try vs tryon, since try includes both tryon, as well as "trying" 
#celebrity names to see if people want to copy signature looks?
#asmr, grwm, vlog have secondary content separate from the objective makeup 
#retail stores to see if people are looking to purchase
#ten - trying to see if people like lists? i.e. "my top ten"

schema["word"] = words


# Adds column, containing means of likes_per_subscriber of entries with the substring
schema['with_string'] = schema["word"].apply(lambda word : get_mean_with_word(df, word, column_to_take_average_of, where_to_look_for_word))

# Adds column, containing means of likes_per_subscriber of entries without the substring
schema['without_string'] = schema["word"].apply(lambda word : get_mean_without_word(df, word, column_to_take_average_of, where_to_look_for_word))


# Adds column, containing means of likes_per_subscriber of entries without the substring
schema['difference'] = schema['with_string']-df["likes_per_subscriber"].mean()


#finding diffrence between with and without substring

schema = schema.sort_values('difference', ascending=False)

# show dataframe 

print(df["likes_per_subscriber"].mean())
display(schema)

0.03705092549069263


,word,with_string,without_string,difference
2,drugstore,0.081252,0.036822,0.044201
1,cheap,0.075682,0.037045,0.038631
0,dupe,0.049351,0.037008,0.012300


In [149]:
#keyword groups
comparison_list = ["unbox","haul","review","try","vs"]
skills_list=["tip","trick","hack","tutorial"]
skincare_list = ["skincare","mask","toner"]#"sunscreen","acne","clean","snail","serum"
product_list=["tint","lipgloss","blush","balm","foundation","lipstick","concealer","eyeshadow","mascara","oil"] #
speed_list=["short","speed","quick","fast","routine"]
#adjective_list=["color","colour","shade","swatch"]
#brand_noun_list=["fenty","dior"]
budget_list=["dupe","cheap","drugstore"] #"budget","affordable"
self_ref_list=["viral","short","popular","fav","best","cute","easy","trend","makeup","beauty"]
acronym_list=["grwm","ootd","asmr"]
kniche_list=["kbeauty","korean"]
#texture_list=["jelly","gel","matte","glitter"]
#season=["summer","winter","fall","valentine","spring"]

In [150]:
def contains_substring(text, substring_list):
    for substring in substring_list:
        if substring.lower() in text.lower():
            return True
    return False

def contains_products(text):
    return 1 * contains_substring(text, product_list)
def contains_budget(text):
    return 1 * contains_substring(text, budget_list)
def contains_self_ref(text):
    return 1 * contains_substring(text, self_ref_list)
def contains_acronym(text):
    return 1 * contains_substring(text, acronym_list)
def contains_kniche(text):
    return 1 * contains_substring(text, kniche_list)
def contains_speed(text):
    return 1 * contains_substring(text, speed_list)
def contains_skills(text):
    return 1 * contains_substring(text, skills_list)
def contains_comparison(text):
    return 1 * contains_substring(text, comparison_list)
def contains_skincare(text):
    return 1 * contains_substring(text, skincare_list)


#skincare and acronym are somewhat poorly performing, but I left them in for now, and we can decide?
#the three groups I would recommend are "comparing_products","skills/teach","speed" and "self_ref". 
#korean is there because it just does very well, would be hard to ignore its effect but doesnt really fit into a group?
#budget is there because...it feels intuitive to mention, just does moderately well

df2["product"] = df2["title"].apply(contains_products)
df2["budget"] = df2["title"].apply(contains_budget)
df2["self_ref"] = df2["title"].apply(contains_self_ref)
df2["acronym"] = df2["title"].apply(contains_acronym)
df2["korean"] = df2["title"].apply(contains_kniche)
df2["speed"] = df2["title"].apply(contains_speed)
df2["skills/teach"] = df2["title"].apply(contains_skills)
df2["skincare"] = df2["title"].apply(contains_skincare)
df2["comparing_products"] = df2["title"].apply(contains_comparison)

In [151]:
display(df2)

,channelUsername,date,title,likes,viewCount,numberOfSubscribers,likes_per_subscriber,views_per_subscriber,duration_in_seconds,days_since_2024_01_01,product,budget,self_ref,acronym,korean,speed,skills/teach,skincare,comparing_products
0,nikkietutorials,2025-01-01T17:03:25.000Z,ICE CREAM lipstick?! 🍦👄,824000.0,10628400.0,14600000.0,0.056438,0.727973,80,366,1,0,0,0,0,0,0,0,0
1,nikkietutorials,2024-09-26T17:09:34.000Z,trying TAPE makeup?! 🤔,468000.0,7355705.0,14600000.0,0.032055,0.503815,59,269,0,0,1,0,0,0,0,0,1
2,nikkietutorials,2024-12-13T17:06:39.000Z,Trying OLD AGE Filter Makeup 👵🏻,329000.0,4324309.0,14600000.0,0.022534,0.296186,66,347,0,0,1,0,0,0,0,0,1
4,nikkietutorials,2024-07-07T17:33:45.000Z,if “trusting the process” was a makeup look 😱,662000.0,8539105.0,14600000.0,0.045342,0.584870,60,188,0,0,1,0,0,0,0,0,0
5,nikkietutorials,2024-11-25T17:47:17.000Z,trying a viral GLITTER mascara… 🤩✨,200000.0,2741522.0,14600000.0,0.013699,0.187775,59,329,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18615,beautyforeverybody,2024-02-04T18:05:27.000Z,💟 These Dupes Made Me Stop Buying ‘The Origina...,258.0,5204.0,22600.0,0.011416,0.230265,23,34,0,1,0,0,0,0,0,0,0
18616,beautyforeverybody,2024-02-07T23:43:52.000Z,💗 Dior Lip Oil Dupe By Catrice Cosmetics,206.0,7658.0,22600.0,0.009115,0.338850,11,37,1,1,0,0,0,0,0,0,0
18617,beautyforeverybody,2024-01-19T21:42:11.000Z,✋🏻 These Dupes Made Me Stop Buying ‘The Origin...,60.0,2781.0,22600.0,0.002655,0.123053,21,18,0,1,0,0,0,0,0,0,0
18618,beautyforeverybody,2024-01-12T18:05:26.000Z,Those Dupes Made Me Stop Buying ‘The Originals...,54.0,2299.0,22600.0,0.002389,0.101726,21,11,0,1,0,0,0,0,0,0,0


In [152]:
df2["budget"].value_counts()

budget
0    15231
1      542
Name: count, dtype: int64

In [153]:
df2["product"].value_counts()

product
0    12589
1     3184
Name: count, dtype: int64

In [154]:
df2["self_ref"].value_counts()

self_ref
0    8072
1    7701
Name: count, dtype: int64

In [155]:
df2["acronym"].value_counts()

acronym
0    15148
1      625
Name: count, dtype: int64

In [156]:
df2["comparing_products"].value_counts()

comparing_products
0    14188
1     1585
Name: count, dtype: int64

In [157]:
df2["skills/teach"].value_counts()

skills/teach
0    14085
1     1688
Name: count, dtype: int64

In [158]:
df2["speed"].value_counts()

speed
0    13230
1     2543
Name: count, dtype: int64

In [159]:
df2["korean"].value_counts()

korean
0    14720
1     1053
Name: count, dtype: int64

In [160]:
keywords = ["product", "skills/teach", "speed", "comparing_products", "self_ref", "budget", "korean"]

for keyword in keywords:
    print(df2.groupby(keyword)["viewCount"].mean())

product
0    413554.299786
1    337533.401696
Name: viewCount, dtype: float64
skills/teach
0    406996.554917
1    324878.527844
Name: viewCount, dtype: float64
speed
0    421895.516553
1    274975.913095
Name: viewCount, dtype: float64
comparing_products
0    412417.731604
1    271014.924290
Name: viewCount, dtype: float64
self_ref
0    471419.618806
1    321470.233476
Name: viewCount, dtype: float64
budget
0    409327.152584
1     85755.664207
Name: viewCount, dtype: float64
korean
0    382845.904552
1    612962.693257
Name: viewCount, dtype: float64


In [161]:
for keyword in keywords:
    print(df2.groupby(keyword)["likes"].mean())

product
0    22085.560728
1    16951.016646
Name: likes, dtype: float64
skills/teach
0    21717.394675
1    15472.545616
Name: likes, dtype: float64
speed
0    22648.101436
1    12730.152969
Name: likes, dtype: float64
comparing_products
0    21618.934734
1    15948.085174
Name: likes, dtype: float64
self_ref
0    25679.727701
1    16195.351123
Name: likes, dtype: float64
budget
0    21610.198345
1     5280.867159
Name: likes, dtype: float64
korean
0    20007.687228
1    35606.842355
Name: likes, dtype: float64


In [162]:
for keyword in keywords:
    print(df2.groupby(keyword)["likes_per_subscriber"].mean())

product
0    0.031722
1    0.063374
Name: likes_per_subscriber, dtype: float64
skills/teach
0    0.038388
1    0.035803
Name: likes_per_subscriber, dtype: float64
speed
0    0.032902
1    0.065213
Name: likes_per_subscriber, dtype: float64
comparing_products
0    0.036444
1    0.053039
Name: likes_per_subscriber, dtype: float64
self_ref
0    0.027635
1    0.049092
Name: likes_per_subscriber, dtype: float64
budget
0    0.038166
1    0.036562
Name: likes_per_subscriber, dtype: float64
korean
0    0.032222
1    0.120437
Name: likes_per_subscriber, dtype: float64


In [163]:
for keyword in keywords:
    print(df2.groupby(keyword)["views_per_subscriber"].mean())

product
0    0.746006
1    1.638080
Name: views_per_subscriber, dtype: float64
skills/teach
0    0.920517
1    0.972530
Name: views_per_subscriber, dtype: float64
speed
0    0.797976
1    1.592563
Name: views_per_subscriber, dtype: float64
comparing_products
0    0.853831
1    1.572848
Name: views_per_subscriber, dtype: float64
self_ref
0    0.683689
1    1.180155
Name: views_per_subscriber, dtype: float64
budget
0    0.914526
1    1.250870
Name: views_per_subscriber, dtype: float64
korean
0    0.795309
1    2.754195
Name: views_per_subscriber, dtype: float64
